# MADRID CENTRAL

## Imports

Different libraries and functions we require to run the code

In [ ]:
import pandas as pd
from matplotlib.path import Path
import numpy as np
import seaborn as sns
import time
from datetime import datetime as dt
import utm
import json
from tqdm import tqdm
import requests
import zipfile
import os
import io
from collections import defaultdict

# folium
import folium
import folium.plugins as plugings

# bokeh
import bokeh.palettes
from bokeh.plotting import figure, show, output_file, save, reset_output
from bokeh.io import output_notebook
from bokeh.models import HoverTool, Legend, ColumnDataSource, Span, ColorBar, DatetimeTickFormatter, CustomJS, Button
from bokeh.models.widgets import Panel, Tabs
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.layouts import layout
from bokeh.transform import linear_cmap


output_notebook()
np.random.seed(42)

## Auxiliar Functions

In [ ]:
def get_color_from_palette(color):
    """ Getting colors for plotting """
    return tuple([int(c * 255) for c in color])

def get_dark_color_from_palette(color):
    """ Getting darker colors for plotting """
    return tuple([int(c * 200) for c in color])

## Constants

Different constants to use through the code.

In [ ]:
DOWNLOAD_DATA = False # Dowload or not the data
GENERATE_TRAFFIC_DATA = False # Generate traffic data

SAVE_PLOTS = False # If true, save the bokeh plots as html. Othewise, show

PALETTE = "colorblind"

DISTRICT_COLORS = [
    get_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 21)
    ]

DISTRICT_DARK_COLORS = [
    get_dark_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 21)
    ]

MADRID_IN_OUT_COLORS = [
    get_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 2)
    ]

MADRID_IN_OUT_DARK_COLORS = [
    get_dark_color_from_palette(c)
    for c in sns.color_palette(PALETTE, 2)
    ]

## 1. Motivation

The purpose of our project is to analyze the real impact of the measure that took place in the city center of Madrid called **Madrid Central**.

To sum up, Madrid Central was an environment measure that entered in force the **30th of November 2018**, it prohibited all polluting vehicles from entering the *"Centro"* district of Madrid. Unfortunately, it was taken down for political and economic reasons the **1st of July 2019**. 

In this study, we aim to evaluate the effectiveness of this vehicle regulation on pollution by analyzing traffic and air quality.

More on **Madrid Central** on the webpage.

<div align="center">
  <img src="https://s03.s3c.es/imag/_v0/864x1252/2/7/c/Madrid-Central_Plano-con-logo.jpg" alt="Madrid Central"/>
</div>

### The Dataset

We use multiple datasets, all of them from the **Open Data platform of the Town hall of Madrid**. 

<div align="center">
  <img src="https://www.zupimages.net/up/22/18/htou.png" alt="Open Data Madrid"/>
</div>


The datasets we use are:

 - **Air Quality Data**
    - [*Calidad del aire. Datos horarios desde 2001* (*Air quality. Hourly data since 2001*)](https://datos.madrid.es/sites/v/index.jsp?vgnextoid=f3c0f7d512273410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD):
    
      This data has in each file different air quality measures from multiple measurement stations. There are 24 stations all around the city of Madrid, with different gases measured in each one of them.

    - [*Calidad del aire. Estaciones de control* (*Air quality. Control stations*)](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=9e42c176313eb410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default):
    
      This is the geodata related to the measurement points. For plotting using Bokeh we had to change the format of how the spatial coordinates are presented.

 - **Traffic Data**
    - [*Tráfico. Histórico de datos del tráfico desde 2013* (*Traffic. Historic traffic data since 2013*)](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=33cb30c367e78410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default):
    
      This data has in separated files the recorded traffic measurements from more than 3000 points, taken every 15 minutes. This is the core dataset we use for our traffic analysis, but, to reduce the size of the data, we preprocess it to only use the average measure on each point per day.
    
    - [*Tráfico. Ubicación de los puntos de medida del tráfico* (*Traffic. Location of traffic measurement points*)](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=ee941ce6ba6d3410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD):
    
      This is the geodata related to the measurement points. For plotting using Bokeh we had to change the format of how the spatial coordinates are presented.

 - **Districts Geodata**

   - [*Distritos municipales de Madrid* (*Municipal districts of Madrid*)](https://datos.madrid.es/sites/v/index.jsp?vgnextoid=7d6e5eb0d73a7710VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD):
   
      The data has geodata of the limits of each of the districts of Madrid


### The Reason of the Dataset

We decided on these two datasets because we wanted to discover if **Madrid Central** achieved the objectives of reducing air pollution in the area, and we also wanted to inspect how effective and how big was the change in the traffic in and out of **Madrid Central**.

Additionally, we desired to analyze the relation between air quality and traffic in the city of Madrid.

### User Experience

While investigating this topic, we saw a lot of articles about **Madrid Central**. Some of them, saying how good and beneficial it was. Others, saying the exact opposite. Sadly, most of the news and articles were quite influenced by political ideas. This is because the measure was imposed by a political party, against the wishes of the opposing political party. Therefore, we feel they were biased, and did not rely on data and science.

We aim to fix this, trying to give an answer to how effective the measure really was, based on data, using visualizations and data analysis. We also want to present this in an informative and interactive way.

## 2. Basic Stats

### Data Processing

The first thing that has to be done in any data related task, is to preprocess and clean the data. In our case, this is very important, as we have as baseline many different files, with a lot of information. We have to reduce the dataset to obtain only the relevant parts.

**Air quality Data Processing**

**Traffic Data Processing**

The data we want to work with is very large, thus we need to download it from the source as it is not possible to upload it to the version control system we use (GitHub). 

In [ ]:
def download_data_traffic():
    """ Download all traffic data from January 2016 (ID=32) until February 2020 (ID=81)
        Some files do not follow the same naming convention, and need repairing.
        The name convention that most files follow is '{num_month}-{num_year}.yaml',
        so everyone will follow that
    """
    FIRST_MONTH_ID = 32
    LAST_MONTH_ID = 81
    DATA_PATH = "data/traffic2"
    
    for month_id in tqdm(range(FIRST_MONTH_ID, LAST_MONTH_ID+1), desc="Downloading data", unit="file"):
        
        # Get month number, from 1 to 12
        current_month = ((month_id - FIRST_MONTH_ID) % 12) + 1
        
        # Get year number, from 2016 to 2021
        current_year = int((month_id - FIRST_MONTH_ID) / 12) + 2016    
               
        # If it has been downloaded already, skip it

        file_path = f"{DATA_PATH}/{current_month:02d}-{current_year}.csv"

        if not os.path.isfile(file_path):

            url = f"https://datos.madrid.es/egob/catalogo/208627-{month_id}-transporte-ptomedida-historico.zip"
            r = requests.get(url)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            zipcsv = z.infolist()[-1]
            
            # Rename file
            zipcsv.filename = file_path
            
            # Extract file
            z.extract(zipcsv)
            

if DOWNLOAD_DATA:
    download_data_traffic()

Before diving into the actual data, we need to contextualize. Madrid is divided into districts. There are *21* one of them, being the area of **Madrid Central** exactly the same as the **Centro district** area (thus the name).

We have a dataset of where the measure of traffic points are located. As expected, they are not evenly distributed. Our first task is to see in which district each traffic measurement point is located.

In [ ]:
traffic_points = pd.read_csv("shared_data/traffic_points/pmed_trafico_03052016.csv", sep=";")
traffic_points.head()

First we need to calculate the correct *utm* for displaying in `bokeh` maps.

In [ ]:
def utm_from_latlon(lat, lon):
    """ From a given lat and lon, calculates the correct UTM coordinates to 
        plot using `bokeh` 
    """
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale

    return x, y

def get_lat_lon_utm(row):
    """ From a row containing the columns 'st_x' and 'st_y' calculates both the lat and lon
        and the correct UTM coordinates to plot using `bokeh`
    """

    # 30 and 'T' is the zone of Madrid
    lat, lon = utm.to_latlon(row["st_x"], row["st_y"], 30, "T")
    
    x, y = utm_from_latlon(lat, lon)

    return pd.Series([lat, lon, x, y])

In [ ]:
traffic_points[["latitude", "longitude", "utm_x", "utm_y"]] = traffic_points.apply(get_lat_lon_utm, axis=1)
traffic_points = traffic_points.rename(columns = {'nombre':'name'})
traffic_points.head()

Then load the districts information to display them in the map.

In [ ]:
with open("shared_data/districts/districts.geojson", "r") as geojson:
    geodata = json.load(geojson)

In [ ]:
df_districts = pd.DataFrame([], columns=["name", "latitude",
                                         "longitude", "utm_x",
                                         "utm_y"])
for district in geodata["features"]:
    # Get district name
    district_name = district["properties"]["NOMBRE"]
    
    # Get district coordinates
    district_coord = district["geometry"]["coordinates"][0]
    df_district = pd.DataFrame(district["geometry"]["coordinates"][0], columns=["st_x", "st_y"])
    df_district["name"] = district_name
    
    # Calculate correct utm
    df_district[["latitude", "longitude", "utm_x", "utm_y"]] = df_district.apply(get_lat_lon_utm, axis=1)
    df_district = df_district.drop(columns=["st_x", "st_y"])
    
    # Append to all districts dataframe
    df_districts = pd.concat([df_districts, df_district]).reset_index(drop=True)


district_name = df_districts["name"].unique()
df_districts

Save in which district is each traffic point.

In [ ]:
traffic_points["district"] = "None"
points = traffic_points[["utm_x", "utm_y"]]

for name in district_name:
    path = Path(df_districts[df_districts["name"] == name][["utm_x", "utm_y"]])
    points_in_path_mask = path.contains_points(points)
    traffic_points.loc[points_in_path_mask, "district"] = name

# Discard the traffic points outside any district of Madrid, as they are outside the city
    
traffic_points = traffic_points.drop(traffic_points[traffic_points["district"] == "None"].index)\
                .reset_index(drop=True)

traffic_points.head()

The next step is to load the datasets for traffic information. This datasets have a lot of rows, as each of the more than 3000 measurement points record mutiple parameters each 15 minutes, so a rough approximation of how many rows each month file has is:

$$ 30(days) \cdot 24(hours) \cdot 4(measures\_per\_hour) \cdot 3000(traffic\_points) = 8640000 $$

And once again, if we take into account that we are using data from 2016 until the end of 2021, a more accurate row count would be:

$$ 4(years) \cdot 365(days) \cdot 24(hours) \cdot 4(measures\_per\_hour) \cdot 3000(traffic\_points) = 42048000 $$

This amount of data (more than 630 million rows) is too much to handle efficiently, and obtain relevant information. To reduce the amount of rows, we decide on keeping the average intensity of traffic (Number of cars) per day in each district. That way, we will have:

$$ 4(years) \cdot 365(days) \cdot 21(number\_districts) = 30660 $$

which is more manageable number, from where we aspire to detect the relevant information in the data. Around 13714 times less data.

In [ ]:
def process_traffic_data(filepath, traffic_points_df):
    """ Function to process each traffic data file. This preoprocess has as objective to reduce
        the dimensionality od the data, only keeping one value per district per day, reducing this
        way the number of rows to handle.
        
        Arguments:
            filepath          -> path to load the csv
            traffic_points_df -> traffic_points dataset (where they are located)
    """
    
    # Load file
    traffic_df = pd.read_csv(filepath, sep=";")
    
    # For god knows why, there is one file that is separated by ',' instead of ';'
    # so we reread the file if it only has one column
    if len(traffic_df.columns) == 1:
        traffic_df = pd.read_csv(filepath, sep=",")
    
    # If the 'idelem' column does not exists, is because is called 'id', so rename column
    if "idelem" not in traffic_df.columns:
        traffic_df = traffic_df.rename(columns = {'id':'idelem'})
    
    # Use only the traffic points for whom we have information 
    traffic_df = traffic_df[traffic_df["idelem"].isin(traffic_points_df["idelem"])]
    
    # Transform date to datime type
    traffic_df["fecha"] = pd.to_datetime(traffic_df["fecha"])
    
    # Get date in separate columns
    traffic_df["day"] = traffic_df["fecha"].dt.day
    traffic_df["month"] = traffic_df["fecha"].dt.month
    traffic_df["year"] = traffic_df["fecha"].dt.year

    # Group by id and date, up to day, and get the average intensity perr traffic point
    traffic_df = traffic_df.groupby(["idelem",
                                     "day",
                                     "month",
                                     "year"]).agg(mean_intensity=("intensidad", "mean")).reset_index()
    
    # Merge with the traffic points to get the district for each point
    traffic_df = traffic_df.merge(traffic_points_df[["idelem", "district"]], on="idelem")
    
    # Save also the traffic points as separated files
    traffic_points_intensity_df = traffic_df.copy()

    # Group by again, to get only one value per district per day
    traffic_df = traffic_df.groupby(["district", "day", "month", "year"]).mean()["mean_intensity"].reset_index()
    
    # Get the date and day of the week for plotting purpose
    traffic_df["date"] = pd.to_datetime(traffic_df[["day", "month", "year"]])
    traffic_df["day_of_week"] = traffic_df["date"].dt.day_name()

    traffic_points_intensity_df["date"] = pd.to_datetime(traffic_points_intensity_df[["day", "month", "year"]])
    traffic_points_intensity_df["day_of_week"] = traffic_points_intensity_df["date"].dt.day_name()

    # Save idelem as integer
    traffic_points_intensity_df["idelem"] = traffic_points_intensity_df["idelem"].astype(int)
    
    return traffic_df, traffic_points_intensity_df

In [ ]:
def load_all_trafic_data(traffic_points_df):
    """ Function to load all trafic data from the data folder,
        after being processed
        
        Arguments:
            traffic_points_df -> traffic_points dataset (where they are located)
    """
    
    DATA_PATH = "data/traffic"
    
    traffic_data = pd.DataFrame([], columns=["district", "date", "day_of_week",
                                             "day", "month", "year", "mean_intensity"])
    
    traffic_points_data = pd.DataFrame([], columns=["district", "date", "day_of_week",
                                             "day", "month", "year", "mean_intensity"])
    
    for filepath in tqdm(os.listdir(DATA_PATH), desc="Processing files", unit="file"):
        traffic_df, traffic_points_intensity_df= process_traffic_data(os.path.join(DATA_PATH, filepath), traffic_points_df)
        
        traffic_data = pd.concat([traffic_data, traffic_df])
        traffic_points_data = pd.concat([traffic_points_data, traffic_points_intensity_df])


    return (traffic_data.sort_values(by=["district", "date"]).reset_index(drop="True"),
            traffic_points_data.sort_values(by=["district", "date", "idelem"]).reset_index(drop="True"))

In [ ]:
df_traffic_path = "shared_data/traffic_intensity.csv"
df_traffic_intensity_path = "data/traffic_points_intensity.csv"


if not GENERATE_TRAFFIC_DATA:
    total_traffic_df = pd.read_csv(df_traffic_path)
    total_traffic_df["date"] = pd.to_datetime(total_traffic_df["date"])

    traffic_points_intensity_df = pd.read_csv(df_traffic_intensity_path)
    traffic_points_intensity_df["date"] = pd.to_datetime(traffic_points_intensity_df["date"])
    traffic_points_intensity_df["idelem"] = traffic_points_intensity_df["idelem"].astype(int)
else:
    total_traffic_df, traffic_points_intensity_df = load_all_trafic_data(traffic_points)

    total_traffic_df.to_csv(df_traffic_path, index=False)
    traffic_points_intensity_df.to_csv(df_traffic_intensity_path, index=False)

In [ ]:
total_traffic_df.head()

In [ ]:
traffic_points_intensity_df.head()

### Exploratory data analysis

**Air quality data exploration**

**Traffic data exploration**

We want to check first how many traffic points are in each district.

In [ ]:
traffic_points.groupby("district").agg(number_traffic_points=("idelem", "count")).sort_values("number_traffic_points").reset_index()

From the table we can see that there is quite a big difference between the different traffic points, being the district with the least number *Barajas* (41) and the district with the biggest number *Chamartín* (357).

This is something we have to take into account, as too few datapoints may result in poor conclusions. Luckly, the *Centro* district has **176** measurement points, which we believe it's enough given the small area (5.23 km²) of this district.

Now let's compare the traffic intensity in each district, to see which districts are busier.

In [ ]:
df_traffic_points = traffic_points.copy()
df_traffic_points = pd.merge(df_traffic_points,
                             traffic_points_intensity_df.groupby("idelem").agg(intensity=("mean_intensity", "mean")).reset_index(),
                             on="idelem")

df_traffic_points.head()

In [ ]:
p = figure(title="Average Intensity per District", x_axis_type="mercator", y_axis_type="mercator",
           height=600, width=600, tools="")

p.axis.visible = False
p.toolbar.logo = None
p.toolbar_location = None


for name in district_name:
    # Districts
    source_dict = dict(utm_x = [[x for x in df_districts[df_districts["name"] == name]["utm_x"]]],
                       utm_y = [[y for y in df_districts[df_districts["name"] == name]["utm_y"]]],
                       name = [name],
                       intensity = [total_traffic_df[total_traffic_df["district"] == name]["mean_intensity"].mean()])

    source = ColumnDataSource(source_dict)
    p.patches(xs="utm_x", ys="utm_y", color=linear_cmap("intensity", "Viridis256", 100, 700), line_width=3, alpha=0.4, 
            source=source, muted=False, muted_alpha=0.1)

    

source = ColumnDataSource(df_traffic_points)

p.circle(x="utm_x", y="utm_y", line_width=1, color=linear_cmap("intensity", "Viridis256", 100, 700),
    source=source, alpha=0.75, size=3)
    

cartodb = get_provider(CARTODBPOSITRON)
p.add_tile(cartodb)

TOOLTIPS = [
    ("District Name", "@name"),
    ("Average Intensity", "@intensity"),
]
p.add_tools(HoverTool(tooltips=TOOLTIPS))

p.background_fill_color = None
p.border_fill_color = None

mapper = linear_cmap(field_name='intensity', palette="Viridis256", low=100, high=700)

color_bar = ColorBar(color_mapper=mapper['transform'], width=8)

p.add_layout(color_bar, 'right')

p = layout(p, sizing_mode='scale_both')

if SAVE_PLOTS:
    output_file("html_plots/average_intensity_districts.html", title="Average Intensity Districts Map")
    save(p)

    reset_output()

    output_notebook()
else:
    show(p)

From this plot, we can see that **Centro** is not the most transited. We do not know yet if it is thanks to the measures, or that it is just not a very transited district. What we do now for sure, and that can be also powered by the measures, the surrounded districts, such as **Arganzuela**, **Retiro** or **Moncloa** are some of the busiest districts in the city, so it will be interesting to take that into account and try to detect if there is a border effect because of **Madrid Central**.  

If we focus more on the traffic points as separated measures instead of districts as a whole, we can observe an expected result. The traffic intensity in the main roads of Madrid are busier than the secondary roads. We can see that, inside **Madrid Central**, the most crowded road is ***Gran Via***, which is the main commercial road in the district. We also appreciate a lot of traffic sorrounding **Madrid Central**, which again may be interesting to investigate if is a normal traffic flow, or is because of the measures. 

In [ ]:
p = figure(title="Traffic intensity through time by district", x_axis_label="Date",
           y_axis_label="Traffic intensity")

fig_lines = []

for name, color in zip(district_name, DISTRICT_COLORS):
    source = ColumnDataSource(total_traffic_df[total_traffic_df["district"] == name])
    l = p.line(x="date", y="mean_intensity", source=source,
               color=color, legend_label=name, visible=True,
               line_width=3, alpha=0.8)
    fig_lines.append(l)
    
p.renderers.extend(fig_lines)

    
p.add_layout(p.legend[0], "right")
p.legend.click_policy = "hide"

p.xaxis.formatter=DatetimeTickFormatter(
        days=['%a %d/%m/%Y'],
        months=['%b %Y'],
        years = ['%Y']
    )

# Hover tooltip
TOOLTIPS = [
    ("District", "@district"),
    ("Intensity", "@mean_intensity"),
    ("Day", "@day_of_week @day/@month/@year")
]
p.add_tools(HoverTool(tooltips=TOOLTIPS, mode="vline"))

# Button
button = Button(
    label="Switch all lines visibility", button_type="success", max_width=100, max_height=50
)
callback = CustomJS(args=dict(lines=fig_lines),
    code="""
    for(var i=0; i<lines.length; i++){
        lines[i].visible = !lines[i].visible;
    }
    """
)
button.js_on_click(callback)


p = layout(p, button, sizing_mode="scale_both")


if SAVE_PLOTS:
    output_file("html_plots/traffict_intensity_time_district.html", title="Traffic intensity through time by district")
    save(p)

    reset_output()

    output_notebook()
else:
    show(p)

## 3. Data analysis

Once we have done our exploratory analysis, it is important to obtain conclusions doing a more in depth analysis of the relevant information obtained.

**Air Quality Data Analysis**

**Traffic Data Analysis**

In our exploratory analysis we visualized a static map. But to see if there is a difference we have to analyze if there is a significant difference between before the regulations and after.

In [ ]:
traffic_points

In [ ]:
df_traffic_points = traffic_points.copy()
df_traffic_points = pd.merge(df_traffic_points,
                             traffic_points_intensity_df.groupby(["idelem", "year", "month"]).agg(intensity=("mean_intensity", "mean")).reset_index(),
                             on="idelem")

df_traffic_points

In [ ]:
map = folium.Map(location=[40.420177, -3.703928], zoom_start=12, tiles='cartodb positron')


heatmap_time_data = defaultdict(list)

for _, row in (df_traffic_points.iterrows()):
    key = str(row["month"]) + " " + str(row["year"])
    heatmap_time_data[key].append([row["latitude"], row["longitude"]])



folium.plugins.HeatMapWithTime(data=list(heatmap_time_data.values()), index=list(heatmap_time_data.keys()) , radius=5, auto_play=True).add_to(map)
map

Ok, this map as informative as we thought it would be. We can see small changes in different districts, but nothing too significative. That is why we will try to carry out a different visualization, in an attempt to display the changes more clearly.

## 4. Genre

### Genre of the data story

We are going to focus on a **Partitioned Poster**, with a special focus on use user interaction. Why? because we believe ...

REVISIT THIS ALL TOGETHER TO DECIDE AND WRITE MORE

### Visual Narrative

**Visual Structuring**: Consistent Visual Platform

**Highlighting**: Close-Ups

**Transition Guidance**: Object Continuity

### Narrative Structure

**Ordering**: User Directed Path

**Interractivity**: Hover Highlighting / Details and Filtering / Selection / Search

**Messaging**: Introductory Text, Captions/Headlines and Summary / Synthesis


## 5. Visualizations

## 6. Discussions

## 7. Contributions

---

# Constants and helper functions

In [ ]:
def utm_from_latlon(lat, lon):
    """ From a given lat and lon, calculates the correct UTM coordinates to 
        plot using `bokeh` 
    """
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale

    return x, y

def get_lat_lon_utm(row):
    """ From a row containing the columns 'st_x' and 'st_y' calculates both the lat and lon
        and the correct UTM coordinates to plot using `bokeh`
    """

    # 30 and 'T' is the zone of Madrid
    lat, lon = utm.to_latlon(row["st_x"], row["st_y"], 30, "T")
    
    x, y = utm_from_latlon(lat, lon)

    return pd.Series([lat, lon, x, y])

In [ ]:
def get_color_from_palette(color):
    """ Getting colors for plotting """
    return tuple([int(c * 255) for c in color])

def get_dark_color_from_palette(color):
    """ Getting darker colors for plotting """
    return tuple([int(c * 200) for c in color])

PALETTE = "colorblind"

In [ ]:
get_color_from_palette(sns.color_palette(PALETTE)[0])

# Madrid Central

We are going to analyze the impact of Madrid Central both from an air quality and a traffic viewpoint.

## Air Quality

In [ ]:
# load air quality stations
df_stations = pd.read_csv('shared_data/air_quality/air_quality_stations.csv')

# load magnitud table
df_magnitud = pd.read_csv('shared_data/air_quality/air_quality_magnitud.csv', sep=';')

# load air quality data
df = pd.read_csv('data/air_quality_data.csv')

# converting Date to datetime type
df["datetime"] = pd.to_datetime(df["datetime"])

# merge with air quality stations
df = pd.merge(df, df_stations, left_on = 'PUNTO_MUESTREO', right_on='punto_muestreo', how='left').drop('PUNTO_MUESTREO', axis=1)

# merge with air quality magnitud
df = pd.merge(df, df_magnitud, left_on = 'MAGNITUD', right_on='magnitud_id', how='left').drop('MAGNITUD', axis=1)

df.head()

In [ ]:
print('AIR QUALITY STATION NAMES:')
print([name for name in df.name.unique()])

In [ ]:
# MAP
# load MC area
cm_points = pd.read_csv('shared_data/districts/central_madrid_points.csv')

points = df_stations[["utm_x", "utm_y"]].values
path = Path(cm_points[["utm_x", "utm_y"]].values)
points_in_path_mask = path.contains_points(points)

df_stations["madrid_central"] = False

df_stations.loc[points_in_path_mask, "madrid_central"] = True

df_stations.head()

In [ ]:
# MAP

# load MC area
cm_points = pd.read_csv('shared_data/districts/central_madrid_points.csv')

points = df_stations[["utm_x", "utm_y"]].values
path = Path(cm_points[["utm_x", "utm_y"]].values)
points_in_path_mask = path.contains_points(points)

# plot map
p = figure(title="Air quality stations in Madrid", x_axis_type="mercator", y_axis_type="mercator")

source = ColumnDataSource(df_stations)
cr = p.circle(x="utm_x", y="utm_y",  size=10, source=source)

cartodb = get_provider(CARTODBPOSITRON)
p.add_tile(cartodb)

p.add_tools(HoverTool(tooltips=[('Name', '@name')], renderers=[cr_in, cr_out]))

# add interactive legend
# legend = Legend(items=[(, [cr_in]), ('OUT of Madrid Central area', [cr_out])], location='center') 
p.legend.click_policy="hide"
# legend.location = "top_left"

p = layout(p, sizing_mode='scale_both')

# output_file("html_plots/air_quality_stations.html", title="Air quality stations")
# save(p)
# reset_output()
# output_notebook()

show(p)

Air quality in different stations

Air quality difference respect previous years

In [ ]:
def get_month_year(aRow):
    return aRow.name.month_name() + ' ' + str(aRow.name.year)

def get_stats_dataframe(station):
    df1 = df[(df.name == station)][['formula','value','datetime']]
    df1 = df1.pivot(index='datetime', columns='formula', values='value').reset_index()
    df1['datetime'] = df1.datetime.dt.floor('D')
    df1['datetime'] = df1['datetime'].apply(lambda dt: dt.replace(day=1))
    tracking_gas = df1.columns.values[1:]
    # cut outliers!!!
    if station == 'Plaza del Carmen':
        df1 = df1[df1.SO2 <500 ]
        df1 = df1[df1.CO < 10 ]
    # homogenization (everything in µg/m^3)
    for aGas in df1.columns.values[1:]:
        unit = df_magnitud[df_magnitud.formula==aGas].unit_per_m3.values[0]
        if ((unit == 'mg') or (unit == '10μg')):
            # we change to 10
            df1[aGas] = df1[aGas] * 100 #*1000 to get it in μg/m3 exactly
            idx = df_magnitud[df_magnitud.formula==aGas].index
            df_magnitud.loc[idx, 'unit_per_m3'] = '10μg'
    # get all stats
    df1_stats = df1.groupby(['datetime']).agg(['mean','std'])
    df1_stats.columns = df1_stats.columns.to_flat_index()
    df1_stats.columns = pd.Index([a+'_'+b for a,b in df1_stats.columns])
    df1_stats['date'] = df1_stats.apply(get_month_year, axis=1)
    for aGas in tracking_gas:
        meanColumn = aGas+'_mean'
        stdColumn = aGas+'_std'
        df1_stats[aGas+'_upper'] = df1_stats[meanColumn]+df1_stats[aGas+'_std']
        df1_stats[aGas+'_lower'] = df1_stats[meanColumn]-df1_stats[aGas+'_std']
    return tracking_gas, df1_stats

def get_bokeh_viz_evolution_over_time(df1_stats, aText, tracking_gas):

    # create annotations for time marks
    startMC = time.mktime(dt(2018, 11, 30, 0, 0, 0).timetuple())*1000
    startMC_span = Span(location=startMC,
                                dimension='height', line_color='black',
                                line_dash='dashed', line_width=2, line_alpha=0.3)

    finesMC = time.mktime(dt(2019, 3, 15, 0, 0, 0).timetuple())*1000
    finesMC_span = Span(location=finesMC,
                                dimension='height', line_color='black',
                                line_dash='dashed', line_width=2, line_alpha=0.3)

    endMC = time.mktime(dt(2019, 7, 1, 0, 0, 0).timetuple())*1000
    endMC_span = Span(location=endMC,
                                dimension='height', line_color='black',
                                line_dash='dashed', line_width=2, line_alpha=0.3)

    cds_stats = ColumnDataSource(data=df1_stats)

    p = figure(
        x_axis_type="datetime",
        width=950,
        height=450,
        title='Evolution of pollutant concentrations over time in '+aText, 
        y_axis_label='Gas Concentration', 
        x_axis_label='Date'
    )

    # create color palette
    colors_gas = dict(zip(tracking_gas,list(bokeh.palettes.brewer['Dark2'][len(tracking_gas)])))

    # add the data of each gas + interactive legend
    lines, circles, bands = {}, {}, {}
    items = [] 
    for aGas in tracking_gas:
        unit = df_magnitud[df_magnitud.formula==aGas].unit_per_m3.values[0]
        # add line of mean
        lines[aGas] = p.line('datetime', aGas+'_mean', source=cds_stats, color = colors_gas[aGas])
        # add dots of mean
        circles[aGas] = p.circle('datetime',aGas+'_mean', source=cds_stats, color=colors_gas[aGas], size=5, alpha=0.5)
        p.add_tools(HoverTool(tooltips=[
            ('Gas',aGas),
            ('Date', '@date'),
            ('Average value', f'@{aGas}_mean {unit}/m3'), 
            ('Standard Deviation', f'@{aGas}_std {unit}/m3')
        ], renderers=[circles[aGas]]))
        # add variance
        bands[aGas] = p.varea(x='datetime', y1=aGas+'_upper', y2=aGas+'_lower', source=cds_stats, fill_alpha=0.1, fill_color=colors_gas[aGas])
        # append legend list
        items.append((f'{aGas} ({unit}/m3)', [lines[aGas], circles[aGas], bands[aGas]]))

    # add legend
    legend = Legend(items=items, location='center') 
    legend.click_policy="hide"
    legend.location = 'top_left'
    p.add_layout(legend)

    # add annotations to plot
    p.add_layout(startMC_span)
    p.add_layout(finesMC_span)
    p.add_layout(endMC_span)
    
    return p

In [ ]:
stations = ['Plaza del Carmen', "Plaza de España", "Castellana", "Retiro", "Méndez Álvaro"]

tabs = []

for station in stations:
    all_tracking_gas, df_stats_station = get_stats_dataframe(station)
    p = get_bokeh_viz_evolution_over_time(df_stats_station, station, all_tracking_gas)

    p = layout(p, sizing_mode='stretch_both')

    tabs.append(Panel(child=p, title=station))

tabs = Tabs(tabs=tabs)

# output_file("html_plots/air_quality_evolution_tabs.html")
# save(tabs)
# reset_output()
# output_notebook()

show(tabs)

## Traffic points

In [ ]:
traffic_points = pd.read_csv("shared_data/traffic_points/pmed_trafico_03052016.csv", sep=";")

traffic_points[["latitude", "longitude", "utm_x", "utm_y"]] = traffic_points.apply(get_lat_lon_utm, axis=1)

traffic_points = traffic_points.rename(columns = {'nombre':'name'})

traffic_points.head()

In [ ]:
with open("shared_data/districts/districts.geojson", "r") as geojson:
    geodata = json.load(geojson)

df_districts = pd.DataFrame([], columns=["name", "latitude",
                                         "longitude", "utm_x",
                                         "utm_y"])
for district in geodata["features"]:
    # Get district name
    district_name = district["properties"]["NOMBRE"]
    
    # Get district coordinates
    district_coord = district["geometry"]["coordinates"][0]
    df_district = pd.DataFrame(district["geometry"]["coordinates"][0], columns=["st_x", "st_y"])
    df_district["name"] = district_name
    
    # Calculate correct utm
    df_district[["latitude", "longitude", "utm_x", "utm_y"]] = df_district.apply(get_lat_lon_utm, axis=1)
    df_district = df_district.drop(columns=["st_x", "st_y"])
    
    # Append to all districts dataframe
    df_districts = pd.concat([df_districts, df_district]).reset_index(drop=True)


district_name = df_districts["name"].unique()
df_districts